# Phân loại bệnh tim sử dụng K-Nearest Neighbors

## 1. Import thư viện

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    roc_auc_score
)

## 2. Đọc dữ liệu

In [2]:
df = pd.read_excel('heart_short.xlsx', sheet_name="data")
df.head()

,t_index,c_index,target
0,145,233,1
1,130,250,1
2,130,204,1
3,130,254,0
4,140,203,0


## 3. Chuẩn bị dữ liệu

In [3]:
X = df[['age','t_index','f_status','c_index']].values
y = df[['target']].values

X = X.astype(float)
y = y.astype(float)

KeyError: "['age', 'f_status'] not in index"

## 4. Chia tập train/test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=16
)

NameError: name 'X' is not defined

## 5. Xây dựng mô hình KNN

In [ ]:
knnModel = KNeighborsClassifier(n_neighbors=5)
knnModel.fit(X_train, y_train.ravel())

## 6. Đánh giá mô hình

In [ ]:
# Dự đoán trên tập test
y_test_predicted = knnModel.predict(X_test)

# Độ chính xác
print("Độ chính xác:", knnModel.score(X_test, y_test))
print("Accuracy score:", accuracy_score(y_test, y_test_predicted))

# Ma trận nhầm lẫn
cf_matrix = confusion_matrix(y_test, y_test_predicted)
print("\nMa trận nhầm lẫn:")
print(cf_matrix)

# Custom metrics
def myscores(smat):
    tp = smat[0][0]
    fp = smat[0][1]
    fn = smat[1][0]
    tn = smat[1][1]
    vprecision = tp/(tp+fp)
    vrecall = tp/(tp+fn)
    vf1 = 2*(vprecision*vrecall)/(vprecision+vrecall)
    return vprecision, vrecall, vf1

print("\nPrecision, Recall, F1:", myscores(cf_matrix))

# Classification report
print("\nBáo cáo phân loại:")
target_names = ['Không bệnh', 'Có bệnh']
print(classification_report(y_test, y_test_predicted, target_names=target_names))

## 7. Vẽ đường ROC

In [ ]:
y_pred_proba = knnModel.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(10,6))
plt.plot(fpr,tpr,'go-',label="AUC="+str(auc))
plt.plot([0,1],[0,1],'r--')
plt.title("AUC & ROC Curve")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=4)
plt.fill_between(fpr, tpr, facecolor='lightgreen', alpha=0.7)
plt.show()

## 8. Lưu mô hình

In [ ]:
pickle.dump(knnModel, open('model_KNN_Heart.sav', 'wb'))

## 9. Dự đoán mẫu mới

In [ ]:
# Load model
loaded_model = pickle.load(open('model_KNN_Heart.sav', 'rb'))

# Nhập thông tin
v1 = float(input('Tuổi (age): '))
v2 = float(input('Chỉ số tim (t_index): '))
v3 = float(input('Tình trạng sức khỏe (f_status): '))
v4 = float(input('Chỉ số mạch (c_index): '))

# Dự đoán
y_pred = loaded_model.predict([[v1,v2,v3,v4]])
print('\nKết quả dự báo bệnh tim:', str(y_pred[0]))

if y_pred[0] == 1:
    print("Kết luận: Bị bệnh tim")
else:
    print("Kết luận: Không bị bệnh tim")